In [ ]:
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms as T
import os

In [ ]:
class CNN2(nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1) 
        self.bn1 = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)  
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(64 * 6 * 6, 256)  
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 7)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.dropout(self.pool(F.relu(self.bn3(self.conv3(x)))))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x
    
model = CNN2()

In [ ]:
classes = {'angry': 0, 'disgusted': 1, 
           'fearful': 2, 'happy': 3, 
           'neutral': 4, 'sad': 5, 
           'surprised': 6}

In [ ]:
def predict(img_path, model, transform):
    model.eval() 
    with torch.no_grad():  
        img = Image.open(img_path).convert('RGB') 
        img = transform(img) 
        img = img.unsqueeze(0) 
        if torch.cuda.is_available(): 
            img = img.cuda()  
            model = model.cuda() 
        y = model(img)
        _, preds = torch.max(y, dim=1)
        predicted_label = preds[0].item()
        return {v: k for k, v in classes}[predicted_label]

In [ ]:
transform = T.Compose([
    T.Resize((48, 48)),  
    T.ToTensor(),  
    T.Normalize([0.5077, 0.5077, 0.5077], [0.2550, 0.2550, 0.2550]) 
])


model = CNN2()  
model.load_state_dict(torch.load('CNN2.pth'))  
model.eval()

# predict(video_path, model, transform)
